In [1]:
from typing import List, Tuple, Any
from datetime import datetime
import os
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
import gym
from gym import wrappers
import matplotlib.pyplot as plt
from pprint import pprint

In [20]:
def get_unix(date: str, time: str) -> int:
    format_date = "%d/%m/%Y"
    format_time = "%H:%M:%S.%f000"
    date_obj = datetime.strptime(date, format_date)
    time_obj = datetime.strptime(time, format_time)
    almost = datetime.combine(date_obj.date(), time_obj.time())
    return int(almost.timestamp())
class Tick:
    def __init__(self, line: str) -> None:
        splitted_line = line.split(',')
        self.date = splitted_line[0]
        self.time = splitted_line[1]
        self.bid = float(splitted_line[2].strip())
        self.ask = float(splitted_line[4].strip())
        self.unix_code = get_unix(splitted_line[0], splitted_line[1])
class Strategy:
    def __init__(self) -> None:
        self.num_strats = 2
        pass 

    def Von_Neuman( self, id: int, prices: List[ float ], tres_hold: float, acumulated_side_bool: bool, cant_part = 8 ) -> int:
        prices = prices[::-1]
        if ( id == 0 ) or ( acumulated_side_bool is False ) :
            up_down = [ ]
            if cant_part < len(prices) :
                for index_groups in range(cant_part - 1) :
                    try:
                        if prices[index_groups * 100] < prices[(index_groups + 1) * 100] :
                            up_down.append(1)
                        elif prices[index_groups * 100] > prices[(index_groups + 1) * 100] :
                            up_down.append(-1)
                        else:
                            up_down.append(0)
                    except IndexError:
                        up_down.append(0)
                try :
                    avg = sum(up_down) / len(up_down)
                    if avg > tres_hold:
                        return -1
                    else:
                        return 0
                except ZeroDivisionError :
                    return 0
            else:
                return 0
        elif ( id > 0 ) and ( acumulated_side_bool is True ) :
            up_down = []
            if cant_part < len(prices):
                for index_groups in range(cant_part - 1) :
                    try:
                        if prices[index_groups * 10] < prices[(index_groups + 1) * 10] :
                            up_down.append(1)
                        elif prices[index_groups * 10] > prices[(index_groups + 1) * 10] :
                            up_down.append(-1)
                        else:
                            up_down.append(0)
                    except IndexError:
                        up_down.append(0)
                try :
                    avg = sum( up_down ) / len( up_down )
                    if -tres_hold <= avg <= tres_hold :
                        return 0
                    elif avg > 0 :
                        return -1
                    else :
                        return 1
                except ZeroDivisionError:
                    return 0
            else:
                return 0
def compute_balance(orders_list: List[Any], tick: Tick) -> float:
    open_balance = 0
    for line in orders_list:
        if len(line) > 2:
            side = line[ 0 ]
            number_units = line[1]
            bid_price, ask_price = line[2], line[3]
            last_bid, last_ask = tick.bid, tick.ask
            profit_loss = 0
            if side == -1:
                profit_loss = number_units * last_bid - number_units * ask_price
            elif side == 1:
                profit_loss = number_units * bid_price - number_units * last_ask
            open_balance += profit_loss
    return open_balance * 1
def compute_drawdown(path_to_use: str, orders_list: List[Any], tick: Tick) -> float:
    if len(orders_list) > 1 :
        last_balance = compute_balance(orders_list, tick)
        balances = []
        with open(path_to_use, 'r') as file_reader:
            for line in file_reader:
                balances.append(float(line.split(':')[6]))
        balances.append(last_balance)
        max_drawdown = 0
        drawdown = 0
        peak = balances[0]
        for balance in balances:
            if balance > peak:
                peak = balance
            drawdown = peak - balance
            if drawdown > max_drawdown:
                max_drawdown = drawdown
        return max_drawdown
    else:
        return 0
def Register(path_to_use: str, line: List[Any]) -> None:
    with open(path_to_use, 'a') as file_in_process:
        for value in line:
            file_in_process.writelines(f'{value}:')
        file_in_process.writelines('\n')        
def file_content(path_to_use: str) -> str:
    if os.path.exists(path_to_use):
        with open(path_to_use, 'r') as file_in_process:
            return file_in_process.read()
    else:
        return ""
def time_signal_detector( current_tick: Tick, previous_tick: Tick, pivot_unix: int ) -> bool:
    current_unix = current_tick.unix_code
    previous_unix = previous_tick.unix_code
    if current_unix <= pivot_unix:
        return True
    elif previous_unix <= pivot_unix < current_unix:
        return False
def State_detector(acumulated_side) -> bool:
    if acumulated_side >= 0 :
          return False
    else :
          return True
time_frames = [7, 14, 21, 28, 35, 42, 49, 56, 63, 70, 77, 84, 91, 
               98, 105, 112, 119, 126, 133, 140, 147, 154, 161, 168, 
               175, 182, 189, 196, 203, 210, 217, 224, 231, 238, 245,
               252, 259, 266, 273, 280, 287, 294, 301, 308, 315, 322,
               329, 336, 343, 350, 357, 364, 371, 378, 385, 392, 399,
               406, 413, 420, 427, 434, 441, 448, 455, 462, 469, 476,
               483, 490, 497, 504, 511, 518, 525, 532, 539, 546, 553,
               560, 567, 574, 581, 588, 595, 602, 609, 616, 623, 630,
               637, 644, 651, 658, 665, 672, 679, 686, 693]
dict_for_tres_hold = {  0.07: 'A',
                        0.08: 'B',
                        0.09: 'C',
                        0.1: 'D',
                        0.15: 'E',
                        0.16: 'F',
                        0.17: 'G',
                        0.18: 'H',
                        0.19: 'I',
                        0.2: 'J',
                        0.25: 'K' }
dict_for_decision =  {  1: 'sell',
                       -1: 'buy'  }
dict_for_time_frames = {}
for i,x in enumerate(time_frames):
    dict_for_time_frames[f'{x}'] = f'P_{i}'
def Process_day(time_frame: str, path_of_current_day: str, day: str, Strat: Strategy ) -> str:
    sum_side_by_day_strat_0 = 0
    sum_side_by_day_strat_1 = 0
    sum_side_by_day_strat_2 = 0
    sum_side_by_day_strat_3 = 0
    sum_side_by_day_strat_4 = 0
    list_for_balance_0 = [ ]
    list_for_balance_1 = [ ]
    prices = [ ]
    index_for_balance = 0
    path_0 = 'Strat_0_'+dict_for_time_frames[time_frame] + '_' +  day
    path_1 = 'Strat_1_'+dict_for_time_frames[time_frame] + '_' +  day
    path_2 = 'Strat_2_'+dict_for_time_frames[time_frame] + '_' +  day
    path_3 = 'Strat_3_'+dict_for_time_frames[time_frame] + '_' +  day
    path_4 = 'Strat_4_'+dict_for_time_frames[time_frame] + '_' +  day
    with open(path_of_current_day, 'r') as current_day:
        current_day_list = current_day.readlines()
        last_hour = int(((current_day_list[-2].split(','))[1].split(':'))[0]) + 1
        current_hour = 0
        int_part, *decimal_part = time_frame.split('.')
        int_part = int(int_part)
        float_part = int(decimal_part[0]) if decimal_part else 0
        increment_minutes = int_part
        increment_seconds = float_part * 6
        current_minute = 0
        current_seconds = 0
        pivot_time = f'{current_hour:02}:{current_minute:02}:{current_seconds:02}.000000000'
        id_0 = 0
        id_1 = 0
        id_2 = 0
        id_3 = 0
        id_4 = 0
        first_move = False
        for index in range(1, len(current_day_list)):
            current_tick = Tick(current_day_list[index])
            previous_tick = Tick(current_day_list[index - 1])
            pivot_unix = get_unix(current_tick.date, pivot_time)
            if time_signal_detector(current_tick, previous_tick, pivot_unix):
                prices.append(np.random.choice([previous_tick.bid, current_tick.ask]))
            else:
                for index_for_strategy in range( Strat.num_strats ) :    
                    if index_for_strategy == 0 :                       
                        selected_tres_hold = np.random.choice([0.07, 0.08, 0.09, 0.1, 0.15, 0.16, 0.17, 0.18, 0.19, 0.2, 0.25])
                        side = Strat.Von_Neuman(id_0, prices, selected_tres_hold, State_detector(sum_side_by_day_strat_0))
                        sum_side_by_day_strat_0 += side
                        if side != 0:
                            list_for_balance_0.append([side, 1, previous_tick.bid, previous_tick.ask])
                            P_L = compute_balance(list_for_balance_0, previous_tick)
                            drawdown = compute_drawdown(path_0, list_for_balance_0, previous_tick)                           
                            Register(path_0, [id_0, dict_for_tres_hold[selected_tres_hold], dict_for_decision[side], 1, previous_tick.bid, previous_tick.ask, P_L, drawdown])
                            id_0 += 1
                        if side == 1 :
                            first_move = True 
                    elif index_for_strategy == 1 :
                        if first_move is True :
                            if side == 1 :
                                new_side = -1 
                            else :
                                new_side = 1 
                            if State_detector( sum_side_by_day_strat_1 + new_side ) :
                                sum_side_by_day_strat_1 += new_side
                                list_for_balance_1.append([new_side, 1, previous_tick.bid, previous_tick.ask])
                                P_L = compute_balance(list_for_balance_1, previous_tick)
                                drawdown = compute_drawdown(path_1, list_for_balance_1, previous_tick)
                                Register(path_1, [id_1,dict_for_tres_hold[selected_tres_hold], dict_for_decision[new_side], 1, previous_tick.bid, previous_tick.ask, P_L, drawdown])
                                id_1 += 1
                    
                current_seconds += increment_seconds
                if current_seconds >= 60:
                    current_seconds -= 60
                    current_minute += 1
                current_minute += increment_minutes
                if current_minute >= 60:
                    extra_hours = current_minute // 60
                    current_minute = current_minute % 60
                    current_hour += extra_hours
                if current_hour >= last_hour:
                    return [ path_0, path_1 ]
                pivot_time = f'{current_hour:02}:{current_minute:02}:{int(current_seconds):02}.000000000'
                prices = []
    return [ path_0, path_1 ]

In [21]:
folder = "C:\\Users\\pvesg\\Desktop\\B-Module\\TXTs"
group_folder_for_results = [
    "C:\\Users\\pvesg\\Desktop\\B-Module\\reuslts\\Von_Neuman", 
    "C:\\Users\\pvesg\\Desktop\\B-Module\\reuslts\\Inverted_VN"]
archives_in_folder = os.listdir( folder )
for archive_name in archives_in_folder:
    if archive_name.endswith('.txt'):
        complete_route = os.path.join(folder, archive_name)
        for time_frame in time_frames:
            print(f'{time_frame} : {complete_route}')
            Strat = Strategy( )
            group_path_of_analized_day = Process_day(str(time_frame), complete_route, archive_name, Strat)
            for path_of_analized_day, folder_for_results in zip(group_path_of_analized_day, group_folder_for_results ) :
                place_path = os.path.join(folder_for_results, path_of_analized_day)
                with open(place_path, 'w') as archive:
                        archive.write(file_content(path_of_analized_day))
                if os.path.exists(path_of_analized_day):
                    os.remove(path_of_analized_day)
                else:
                    pass

7 : C:\Users\pvesg\Desktop\B-Module\TXTs\01-02-2023.txt
14 : C:\Users\pvesg\Desktop\B-Module\TXTs\01-02-2023.txt
21 : C:\Users\pvesg\Desktop\B-Module\TXTs\01-02-2023.txt
28 : C:\Users\pvesg\Desktop\B-Module\TXTs\01-02-2023.txt


KeyboardInterrupt: 

In [2]:
from typing import List, Any
import numpy as np
import pandas as pd
from datetime import datetime, timedelta

In [6]:
def get_unix(date: str, time: str) -> int:
    format_date = "%d/%m/%Y"
    format_time = "%H:%M:%S.%f000"
    date_obj = datetime.strptime(date, format_date)
    time_obj = datetime.strptime(time, format_time)
    almost = datetime.combine(date_obj.date(), time_obj.time())
    return int(almost.timestamp())
class Tick:
    def __init__(self, line: str) -> None:
        splitted_line = line.split(',')
        self.date = splitted_line[0]
        self.time = splitted_line[1]
        self.bid = float(splitted_line[2].strip())
        self.ask = float(splitted_line[4].strip())
        self.unix_code = get_unix(splitted_line[0], splitted_line[1])
class Strategy:
    def __init__(self) -> None:
        self.num_strats = 4
        pass 

    def Von_Neuman( self, id: int, prices: List[ float ], tres_hold: float, acumulated_side_bool: bool, cant_part = 8 ) -> int:
        prices = prices[::-1]
        if ( id == 0 ) or ( acumulated_side_bool is False ) :
            up_down = [ ]
            if cant_part < len(prices) :
                for index_groups in range(cant_part - 1) :
                    try:
                        if prices[index_groups * 100] < prices[(index_groups + 1) * 100] :
                            up_down.append(1)
                        elif prices[index_groups * 100] > prices[(index_groups + 1) * 100] :
                            up_down.append(-1)
                        else:
                            up_down.append(0)
                    except IndexError:
                        up_down.append(0)
                try :
                    avg = sum(up_down) / len(up_down)
                    if avg > tres_hold:
                        return -1
                    else:
                        return 0
                except ZeroDivisionError :
                    return 0
            else:
                return 0
        elif ( id > 0 ) and ( acumulated_side_bool is True ) :
            up_down = []
            if cant_part < len(prices):
                for index_groups in range(cant_part - 1) :
                    try:
                        if prices[index_groups * 10] < prices[(index_groups + 1) * 10] :
                            up_down.append(1)
                        elif prices[index_groups * 10] > prices[(index_groups + 1) * 10] :
                            up_down.append(-1)
                        else:
                            up_down.append(0)
                    except IndexError:
                        up_down.append(0)
                try :
                    avg = sum( up_down ) / len( up_down )
                    if -tres_hold <= avg <= tres_hold :
                        return 0
                    elif avg > 0 :
                        return -1
                    else :
                        return 1
                except ZeroDivisionError:
                    return 0
            else:
                return 0
    def mean_proximity(self, prices) -> int:
        try:
            if not prices:
                raise ValueError("La lista 'prices' no debe estar vacía")
            
            median = np.mean(prices)
            up_diff = np.max(prices)
            down_diff = np.min(prices)

            if abs(median - up_diff) > abs(median - down_diff):
                return 1
            elif abs(median - up_diff) < abs(median - down_diff):
                return -1
            else:
                return 0
        except Exception as e:
            return 0  # Valor por defecto en caso de error
    def above_under_mean(self, price :float, tres_hold: float, first_price: float) -> int:
        try:
            if not price:
                raise ValueError("La lista 'prices' no debe estar vacía")
            threshold_amount = tres_hold * first_price
            print('in')
            if price < first_price - threshold_amount:
                return 1
            elif price > first_price + threshold_amount:
                return -1
            else:
                return 0
        except Exception as e:
            return 0  
                
def compute_balance(orders_list: List[Any], tick: Tick) -> float:
    open_balance = 0
    for line in orders_list:
        if len(line) > 2:
            side = line[ 0 ]
            number_units = line[1]
            bid_price, ask_price = line[2], line[3]
            last_bid, last_ask = tick.bid, tick.ask
            profit_loss = 0
            if side == -1:
                profit_loss = number_units * last_bid - number_units * ask_price
            elif side == 1:
                profit_loss = number_units * bid_price - number_units * last_ask
            open_balance += profit_loss
    return open_balance * 1
def compute_drawdown(path_to_use: str, orders_list: List[Any], tick: Tick) -> float:
    if len(orders_list) > 1 :
        last_balance = compute_balance(orders_list, tick)
        balances = []
        with open(path_to_use, 'r') as file_reader:
            for line in file_reader:
                balances.append(float(line.split(':')[6]))
        balances.append(last_balance)
        max_drawdown = 0
        drawdown = 0
        peak = balances[0]
        for balance in balances:
            if balance > peak:
                peak = balance
            drawdown = peak - balance
            if drawdown > max_drawdown:
                max_drawdown = drawdown
        return max_drawdown
    else:
        return 0
def Register(path_to_use: str, line: List[Any]) -> None:
    with open(path_to_use, 'a') as file_in_process:
        for value in line:
            file_in_process.writelines(f'{value}:')
        file_in_process.writelines('\n')        
def file_content(path_to_use: str) -> str:
    if os.path.exists(path_to_use):
        with open(path_to_use, 'r') as file_in_process:
            return file_in_process.read()
    else:
        return ""
def time_signal_detector( current_tick: Tick, previous_tick: Tick, pivot_unix: int ) -> bool:
    current_unix = current_tick.unix_code
    previous_unix = previous_tick.unix_code
    if current_unix <= pivot_unix:
        return True
    elif previous_unix <= pivot_unix < current_unix:
        return False
def State_detector(acumulated_side) -> bool:
    if acumulated_side >= 0 :
          return False
    else :
          return True
time_frames = [7, 14, 21, 28, 35, 42, 49, 56, 63, 70, 77, 84, 91, 
               98, 105, 112, 119, 126, 133, 140, 147, 154, 161, 168, 
               175, 182, 189, 196, 203, 210, 217, 224, 231, 238, 245,
               252, 259, 266, 273, 280, 287, 294, 301, 308, 315, 322,
               329, 336, 343, 350, 357, 364, 371, 378, 385, 392, 399,
               406, 413, 420, 427, 434, 441, 448, 455, 462, 469, 476,
               483, 490, 497, 504, 511, 518, 525, 532, 539, 546, 553,
               560, 567, 574, 581, 588, 595, 602, 609, 616, 623, 630,
               637, 644, 651, 658, 665, 672, 679, 686, 693]
dict_for_tres_hold = {  0.07: 'A',
                        0.08: 'B',
                        0.09: 'C',
                        0.1: 'D',
                        0.15: 'E',
                        0.16: 'F',
                        0.17: 'G',
                        0.18: 'H',
                        0.19: 'I',
                        0.2: 'J',
                        0.25: 'K' }
dict_for_decision =  {  1: 'sell',
                       -1: 'buy'  }
dict_for_time_frames = {}
for i,x in enumerate(time_frames):
    dict_for_time_frames[f'{x}'] = f'P_{i}'

def Process_day(time_frame: str, path_of_current_day: str, day: str, Strat: Strategy) -> List[str]:
    # Inicialización de variables
    sum_side_by_day_strats = [0] * 4
    list_for_balances = [[] for _ in range(4)]
    paths = [f'Strat_{i}_{dict_for_time_frames[time_frame]}_{day}' for i in range(4)]
    prices = [ ]
    first_move = False
    ids = [0] * 4
    side = [0] * 4
    int_part, *decimal_part = time_frame.split('.')
    int_part = int(int_part)
    float_part = int(decimal_part[0]) if decimal_part else 0
    time_increment = timedelta(minutes=int_part, seconds=float_part * 6)
    with open(path_of_current_day, 'r') as current_day:
        current_day_list = current_day.readlines()
        last_hour = int(current_day_list[-2].split(',')[1].split(':')[0]) + 1
        current_time = datetime.strptime("00:00:00.000", "%H:%M:%S.%f")
        first_tick = Tick(current_day_list[0])
        
        for index in range(1, len(current_day_list)):

            current_tick = Tick(current_day_list[index])
            previous_tick = Tick(current_day_list[index - 1])

            if not current_tick or not previous_tick:
                continue
            if True :
                        side[3] = Strat.above_under_mean(price = np.random.choice([current_tick.bid,current_tick.ask]), tres_hold = 1.4, first_price= (first_tick.bid + first_tick.ask)/2)  
                        if State_detector(sum_side_by_day_strats[3]+side[3]) and side[3]!= 0:
                            sum_side_by_day_strats[3] += side[3]
                            list_for_balances[3].append([side[3], 1, previous_tick.bid, previous_tick.ask])
                            P_L = compute_balance(list_for_balances[3], previous_tick)
                            drawdown = compute_drawdown(paths[3], list_for_balances[3], previous_tick)
                            Register(paths[3], [ids[3], dict_for_decision[side[3]], 1, previous_tick.bid, previous_tick.ask, P_L, drawdown])
                            ids[3] += 1      
            pivot_unix = get_unix(current_tick.date, current_time.strftime("%H:%M:%S.%f"))
            if time_signal_detector(current_tick, previous_tick, pivot_unix):
                prices.append(np.random.choice([previous_tick.bid, current_tick.ask]))
            else:
                selected_tres_hold = np.random.choice([0.07, 0.08, 0.09, 0.1, 0.15, 0.16, 0.17, 0.18, 0.19, 0.2, 0.25])
                side[0] = Strat.Von_Neuman(ids[0], prices, selected_tres_hold, State_detector(sum_side_by_day_strats[0]))

                for strat_id in range(Strat.num_strats-1):
                    if strat_id == 0:
                        if side[0] != 0:
                            sum_side_by_day_strats[0] += side[0]
                            list_for_balances[0].append([side[0], 1, previous_tick.bid, previous_tick.ask])
                            P_L = compute_balance(list_for_balances[0], previous_tick)
                            drawdown = compute_drawdown(paths[0], list_for_balances[0], previous_tick)
                            Register(paths[0], [ids[0], dict_for_tres_hold[selected_tres_hold], dict_for_decision[side[0]], 1, previous_tick.bid, previous_tick.ask, P_L, drawdown])
                            ids[0] += 1
                        if side[0] == 1:
                            first_move = True

                    elif strat_id == 1:
                        if first_move :
                            if side[0] == 1:
                                side[1] = -1 
                            else:
                                side[1] = 1                   
                            if State_detector(sum_side_by_day_strats[1] + side[1]):
                                sum_side_by_day_strats[1] += side[1]
                                list_for_balances[1].append([side[1], 1, previous_tick.bid, previous_tick.ask])
                                P_L = compute_balance(list_for_balances[1], previous_tick)
                                drawdown = compute_drawdown(paths[1], list_for_balances[1], previous_tick)
                                Register(paths[1], [ids[1], dict_for_tres_hold[selected_tres_hold], dict_for_decision[side[1]], 1, previous_tick.bid, previous_tick.ask, P_L, drawdown])
                                ids[1] += 1
                    elif strat_id == 2 :
                        side[2] = Strat.mean_proximity(prices = prices) 
                        if State_detector(sum_side_by_day_strats[2]+side[2]) and side[2]!=0:
                            sum_side_by_day_strats[2] += side[2]
                            list_for_balances[2].append([side[2], 1, previous_tick.bid, previous_tick.ask])
                            P_L = compute_balance(list_for_balances[2], previous_tick)
                            drawdown = compute_drawdown(paths[2], list_for_balances[2], previous_tick)
                            Register(paths[2], [ids[2], dict_for_decision[side[2]], 1, previous_tick.bid, previous_tick.ask, P_L, drawdown])
                            ids[2] += 1
                   
                current_time += time_increment
                if current_time.hour >= last_hour:
                    return paths[:Strat.num_strats]

                prices = []

    return paths[:Strat.num_strats]


In [7]:
folder = "E:\B-Module\TXTss"
group_folder_for_results = [
    "E:\B-Module\results\Von_Neuman" , 
    "E:\B-Module\results\Inverted_VN",
    "E:\B-Module\results\mean_proximity" ,
    "E:\B-Module\results\above_under_mean"
    ]
archives_in_folder = os.listdir( folder )
for archive_name in archives_in_folder:
    if archive_name.endswith('.txt'):
        complete_route = os.path.join(folder, archive_name)
        for time_frame in time_frames:
            print(f'{time_frame} : {complete_route}')
            Strat = Strategy( )
            group_path_of_analized_day = Process_day(str(time_frame), complete_route, archive_name, Strat)
            for path_of_analized_day, folder_for_results in zip(group_path_of_analized_day, group_folder_for_results ) :
                place_path = os.path.join(folder_for_results, path_of_analized_day)
                with open(place_path, 'w') as archive:
                        archive.write(file_content(path_of_analized_day))
                if os.path.exists(path_of_analized_day):
                    os.remove(path_of_analized_day)
                else:
                    pass

7 : E:\B-Module\TXTss\01-02-2023.txt
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in


KeyboardInterrupt: 